<a href="https://colab.research.google.com/github/yilinearity/SimpleHMO/blob/main/HMO_Calculation_v1_3_GColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Solving Simple Huckel Systems**

This script is formally suitable for **pi-conjugated hydrocarbon** only. However, by replacing the **Hii (Coulomb integrals)** and **Hij (resonance integrals)** in the Huckel matrix with suitable values, heteoatom-containing systems can also be considered. Examples are given below in Step 2-1 or Step 2-2.

For the **Hii** and **Hij** values for heteroatoms: see F. A. Van-Catledge, *J. Org. Chem.*, **1980**, *45*, 4801–4802; or W. P. Purcell, J. A. Singer, *J. Chem. Eng. Data*, **1967**, *12*, 235–246.

The orbital energy (eigen energies) and orbital coefficients (eigen vectors) are calculated.

<font color='blue'>Click on the **run** icon (on the left side of the code) to run each cell.</font>

## Step 1

The cell below generates the Huckel matrix from the SMILES string using RDKit's `rdmolops.GetAdjacencyMatrix` module. Make sure that the RDKit has been installed (run the first cell below). The SMILES can be, for instance, obtained from ChemDraw or Wikipedia.

This step can be skipped if you want to generate the Huckel matrix from scratch.

- The output Matrix 1 has <font color='red'>**Hii = 0**</font> and <font color='red'>**Hij = 1**</font> . Use this matrix in Step 2-1.
- The output Matrix 2 has <font color='red'>**Hii = a (alpha)**</font>  and <font color='red'>**Hij = b (beta)**</font>. Use this matrix in Step 2-2.

<br>

---
MIT License Copyright (c) 2023 Yi-Lin Wu (yilinearity@gmail.com) <br>
Full text of the MIT License can be found at: https://opensource.org/licenses/MIT

In [ ]:
# Only run this cell ONCE at the begining of use
!pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolops

# Define a function to generate the adjacency matrix from a SMILES string
def smiles_to_adjacency_matrix(smiles):
    """
    Converts a SMILES string to an adjacency matrix using RDKit.

    Args:
        smiles: The SMILES string of the molecule.

    Returns:
        AC_str: The adjacency matrix as a formatted string.
        AC_modified_str: The modified adjacency matrix as a formatted string.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        AC = rdmolops.GetAdjacencyMatrix(mol)
        num_atoms = len(AC)

        # Modify the adjacency matrix
        AC_modified = []
        for i in range(num_atoms):
            row = []
            for j in range(num_atoms):
                if i == j:
                    row.append("a")
                elif AC[i, j] == 1:
                    row.append("b")
                else:
                    row.append("0")
            AC_modified.append(row)

        AC_str = "[\n" + "\n".join(["  [" + ", ".join(map(str, row)) + "]," for row in AC]) + "\n]"
        AC_modified_str = "[\n" + "\n".join(["  [" + ", ".join(row) + "]," for row in AC_modified]) + "\n]"
        return AC_str, AC_modified_str
    except Exception as e:
        print(f"Error converting SMILES to adjacency matrix: {e}")
        return None, None

# SMILES string
smiles = "c1ccccc1"

# Convert SMILES to adjacency matrix
adjacency_matrix, modified_adjacency_matrix = smiles_to_adjacency_matrix(smiles)

if adjacency_matrix is not None:
    print("Matrix 1:")
    print(adjacency_matrix)

    print("\nMatrix 2:")
    print(modified_adjacency_matrix)
else:
    print("Conversion failed.")


## Step 2-1

Replace H matrix below by **Matrix 1** from Step 1. Make sure it is enclosed in `np.array()`

In [ ]:
import numpy as np

# Define the Huckel matrix

## Examples
## 1,3-Butadiene, Huckel MO energies: -1.62, -0.62, 0.62, 1.62
# H = np.array([[0, 1, 0, 0],[1, 0, 1, 0],[0, 1, 0, 1],[0, 0, 1, 0]])
## Cyclobutadiene, Huckel MO energies: -2.00, -0.00, 0.00, 2.00
# H = np.array([[0, 1, 0, 1], [1, 0, 1, 0], [0, 1, 0, 1], [1, 0, 1, 0]])
## Benzene, Huckel MO energies: -2.00, -1.00, -1.00, 1.00, 1.00, 2.00
# H = np.array([[0, 1, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0], [0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 0]])
## Ethene, Huckel MO energies: -1.00, 1.00
# H = np.array([[0, 1], [1, 0]])
## Formaldehyde, Huckel model energies: -0.68, 1.65
# H = np.array([[0, 1.06], [1.06, 0.97]])
## Thioformaldehyde, Huckel model energies: -0.61, 1.07
# H = np.array([[0, 0.81], [0.81, 0.46]])
## Formic acid, Huckel model energies: -0.79, 1.48, 2.37
# H = np.array([[0, 1.06,  0.66], [1.06, 0.97, 0], [0.66, 0, 2.09]])
## Thioformic acid (thio form), Huckel model energies: -0.86, 1.07, 1.87
# H = np.array([[0, 1.06,  0.69], [1.06, 0.97, 0], [0.69, 0, 1.11]])

H = np.array([
 [0, 1, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 0],
 [0, 1, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 1, 0],
])

# Validate the matrix for symmetry
if not np.allclose(H, H.T, atol=0.0001):
    raise ValueError("Input matrix must be symmetric.")

# Calculate eigenvalues and eigenvectors
energies, vectors = np.linalg.eigh(H)
energies = np.around(energies, decimals=3)
eigen_vectors = list(vectors.T)
energy_eigens = {}

# Print all energies
print("Huckel MO energies:", ", ".join([f"{energy:.2f}" for energy in energies]))
print("\n")

for e, vec in zip(energies, eigen_vectors):
    vectors = energy_eigens.get(e, [])
    vectors.append(vec)
    energy_eigens[e] = vectors

# Print the individual orbital energies and coefficients
for energy, eigen_vectors in energy_eigens.items():
    print(f"Energy Level: {energy:.2f}")
    for i, coefficients in enumerate(eigen_vectors):
        print(f"Normalized orbital coefficients: {', '.join([f'{c:.2f}' for c in coefficients])}")

## Step 2-2

Replace H matrix below by **Matrix 2** from Step 1. Make sure it is enclosed in `sp.Matrix()`

In [ ]:
import sympy as sp

# Define symbolic variables
a, b = sp.symbols('a b')

# Define the Huckel matrix

## Examples
## 1,3-Butadiene, Huckel MO energies: a - sqrt(5)*b/2 - b/2, a - sqrt(5)*b/2 + b/2, a - b/2 + sqrt(5)*b/2, a + b/2 + sqrt(5)*b/2
# H = sp.Matrix([[a, b, 0, 0], [b, a, b, 0], [0, b, a, b], [0, 0, b, a]])
## Cyclobutadiene, Huckel MO energies: a - 2*b, a, a, a + 2*b
# H = sp.Matrix([[a, b, 0, b], [b, a, b, 0], [0, b, a, b], [b, 0, b, a]])
## Benzene, Huckel MO energies: a - 2*b, a - b, a - b, a + b, a + b, a + 2*b
# H = sp.Matrix([[a, b, 0, 0, 0, b], [b, a, b, 0, 0, 0], [0, b, a, b, 0, 0], [0, 0, b, a, b, 0], [0, 0, 0, b, a, b], [b, 0, 0, 0, b, a]])
## Ethene, Huckel MO energies: a - b, a + b
# H = sp.Matrix([[a, b],[b, a]])
## Formaldehyde, Huckel model energies: a-0.68b, a+1.65b
# H = sp.Matrix([[a, 1.06*b],[1.06*b, a+0.97*b]])
## Thioformaldehyde, Huckel model energies: a-0.61b, a+1.07b
# H = sp.Matrix([[a, 0.81*b], [0.81*b, a+0.46*b]])
## Formic acid, Huckel model energies: complicated expression
# H = sp.Matrix([[a, 1.06*b,  0.66*b], [1.06*b, a+0.97*b, 0], [0.66*b, 0, a+2.09*b]])
## Thioformic acid (thio form), Huckel model energies: complicated expression
# H = sp.Matrix([[a, 1.06*b,  0.69*b], [1.06*b, a+0.97*b, 0], [0.69*b, 0, a+1.11*b]])

H = sp.Matrix([
 [a, b, 0, 0, 0, b],
 [b, a, b, 0, 0, 0],
 [0, b, a, b, 0, 0],
 [0, 0, b, a, b, 0],
 [0, 0, 0, b, a, b],
 [b, 0, 0, 0, b, a],
])

# Calculate eigenvalues and eigenvectors symbolically
eigenvalues = H.eigenvals()
eigenvectors = H.eigenvects()

# Create a dictionary to count eigenvalue occurrences
eigenvalue_counts = {val: mult for val, mult in eigenvalues.items()}

# Sort eigenvalues by their expressions as symbolic expressions
sorted_eigenvalues = sorted(eigenvalues.keys(), key=lambda x: x.subs({a: 0, b: 1}), reverse=False)

# Step 1: Print all eigenvalues with degeneracy
print("Huckel MO energies:")
output = []
for eigenvalue in sorted_eigenvalues:
    degeneracy = eigenvalue_counts[eigenvalue]
    output.extend([str(eigenvalue)] * degeneracy)
print(', '.join(output))
print()

# Step 2: Normalize and print eigenvectors for each energy level
for eigenvalue in sorted_eigenvalues:
    degeneracy = eigenvalue_counts[eigenvalue]
    print("\nEnergy Level:", eigenvalue)
    eigenvectors_for_eigenvalue = [evect for val, mult, evect in eigenvectors if val == eigenvalue]
    for i, eigenvector in enumerate(eigenvectors_for_eigenvalue):
        normalized_eigenvectors = []
        for ev in eigenvector:
            normalized_eigenvector = ev / ev.norm()
            print("Normalized orbital coefficients:", end=' ')
            for component in normalized_eigenvector:
                print(component, end=' ')
            print()
